# Fusion 360 Gewindetool – XML-Editor mit Jupyter Notebook

Dieses Projekt hilft dir, benutzerdefinierte Gewinde für Autodesk Fusion 360 einfach in eine XML-Datei einzutragen.
Es verwendet ein interaktives Jupyter Notebook mit `ipywidgets`, um Eingaben auf Deutsch zu erfassen und korrekt in das englischsprachige XML-Format zu übertragen.

In [87]:
from IPython.display import display, HTML
import ipywidgets as widgets
from ipywidgets import Layout, HBox, VBox
import xml.etree.ElementTree as ET
import subprocess
import datetime
import threading
import time
import xml.dom.minidom
import shutil
from pathlib import Path
import configparser

# --- Buttons für Export, Import, venv ---
export_button = widgets.Button(description='Get XML from Fusion')
import_button = widgets.Button(description='Patch Fusion XML')
venv_button = widgets.Button(description='Rebuild .venv')
status_label = widgets.Label(value='⏳ Noch kein Export oder Import ausgeführt.')
control_buttons = HBox([export_button, import_button, venv_button])

config_file = Path('./config_fusion_threaddata_path.ini')
config = configparser.ConfigParser()

default_fusion_path = "/Users/apos/Library/Application Support/Autodesk/webdeploy/production/e70c239bb1d26fec5aba0c1faee6762025157baa/Autodesk Fusion.app/Contents/Libraries/Applications/Fusion/Fusion/Server/Fusion/Configuration/ThreadData"

# Wenn Konfigurationsdatei nicht existiert → anlegen
if not config_file.exists():
    config['Fusion'] = {'ThreadDataPath': default_fusion_path}
    with open(config_file, 'w') as f:
        config.write(f)
    print(f"⚠️ Konfigurationsdatei {config_file} wurde neu angelegt.")

# Konfigurationsdatei einlesen
config.read(config_file)
fusion_path_str = config['Fusion']['ThreadDataPath']
fusion_path = Path(fusion_path_str)

# Existenz prüfen
if not fusion_path.exists():
    status_label.value = f"⚠️ Warnung: Der Pfad {fusion_path} existiert nicht!"
else:
    status_label.value = f"✅ Pfad geladen: {fusion_path}"

local_file = Path("./data/AstroISOmetric.xml")
backup_file = local_file.with_suffix('.xml.bak')

def export_xml():
    shutil.copy(fusion_path / 'AstroISOmetric.xml', backup_file)
    shutil.copy(fusion_path / 'AstroISOmetric.xml', local_file)
    print("✅ Datei aus Fusion-Ordner ins data-Verzeichnis kopiert")

def import_xml():
    shutil.copy(fusion_path / 'AstroISOmetric.xml', fusion_path / 'AstroISOmetric.xml.bak')
    shutil.copy(local_file, fusion_path / 'AstroISOmetric.xml')
    print("✅ Datei aus data-Verzeichnis zurück ins Fusion-Programm kopiert")


# Sprachumschalter und Übersetzungen
current_language = {'lang': 'EN'}
selected_designation = {'value': None}

labels = {
    'DE': {
        'headline': 'Fusion 360 Gewindetool – XML-Editor mit Jupyter Notebook',
        'thread_dropdown': 'Vorhandene Gewinde:',
        'size': 'Nenndurchmesser (Size):',
        'designation_de': 'Custom Thread designation - Freitext (CTD):',
        'designation_en': 'Technische Bezeichnung (ThreadDesignation) z.B. M56x0.75:',
        'pitch': 'Steigung (Pitch):',
        'ext_6g': '<b>External 6g</b>',
        'int_6H': '<b>Internal 6H</b>',
        'ext_4g6g': '<b>External 4g6g</b>',
        'save': 'Speichern',
        'save_new': 'Als neu speichern',
        'delete': 'Löschen',
        'export': 'Get XML from Fusion',
        'import': 'Patch Fusion XML',
        'venv': 'Rebuild .venv',
        'status': '⏳ Noch kein Export oder Import ausgeführt.'
    },
    'EN': {
        'headline': 'Fusion 360 Thread Tool – XML Editor with Jupyter Notebook',
        'thread_dropdown': 'Existing Threads:',
        'size': 'Nominal Diameter (Size):',
        'designation_de': 'Custom Thread designation (free text, CTD):',
        'designation_en': 'Technical designation (ThreadDesignation) e.g. M56x0.75:',
        'pitch': 'Pitch:',
        'ext_6g': '<b>External 6g</b>',
        'int_6H': '<b>Internal 6H</b>',
        'ext_4g6g': '<b>External 4g6g</b>',
        'save': 'Save',
        'save_new': 'Save as New',
        'delete': 'Delete',
        'export': 'Get XML from Fusion',
        'import': 'Patch Fusion XML',
        'venv': 'Rebuild .venv',
        'status': '⏳ No export or import done yet.'
    }
}

# Headline Widget
headline = widgets.HTML(value=f"<h1>{labels[current_language['lang']]['headline']}</h1>")
# display(headline)

# Sprachumschalter-Button mit Flaggen
toggle_lang_button = widgets.Button(description='🇬🇧 / 🇩🇪')

def update_labels():
    lang = current_language['lang']
    headline.value = f"<h1>{labels[lang]['headline']}</h1>"
    thread_dropdown.description = labels[lang]['thread_dropdown']
    size.description = labels[lang]['size']
    designation_de.description = labels[lang]['designation_de']
    designation_en.description = labels[lang]['designation_en']
    pitch.description = labels[lang]['pitch']
    label_ext_6g.value = labels[lang]['ext_6g']
    label_int_6H.value = labels[lang]['int_6H']
    label_ext_4g6g.value = labels[lang]['ext_4g6g']
    save_button.description = labels[lang]['save']
    save_new_button.description = labels[lang]['save_new']
    delete_button.description = labels[lang]['delete']
    export_button.description = labels[lang]['export']
    import_button.description = labels[lang]['import']
    venv_button.description = labels[lang]['venv']
    status_label.value = labels[lang]['status']

def on_toggle_lang_clicked(b):
    if current_language['lang'] == 'EN':
        current_language['lang'] = 'DE'
        toggle_lang_button.description = '🇬🇧 Switch to English'
    else:
        current_language['lang'] = 'EN'
        toggle_lang_button.description = '🇩🇪 Auf Deutsch umschalten'
    update_labels()

toggle_lang_button = widgets.Button(description='🇩🇪 Auf Deutsch umschalten')
toggle_lang_button.on_click(on_toggle_lang_clicked)



# CSS hacks
display(HTML("<style>.highlighted select { border: 3px solid lightgreen !important; }</style>"))
headline = widgets.HTML(value="<h1>Fusion 360 Gewindetool – XML-Editor mit Jupyter Notebook</h1>")
display(headline)


# Feste XML-Datei
xml_path = 'data/AstroISOmetric.xml'

def remove_blank_lines(xml_string):
    lines = xml_string.split('\n')
    non_empty_lines = [line for line in lines if line.strip() != '']
    return '\n'.join(non_empty_lines)

def pretty_print_xml(path):
    with open(path, 'r') as f:
        xml_string = f.read()
    parsed = xml.dom.minidom.parseString(xml_string)
    with open(path, 'w') as f:
        f.write(parsed.toprettyxml(indent="  "))

            
def clean_invalid_threads(path):
    tree = ET.parse(path)
    root = tree.getroot()
    to_remove = []
    for ts in root.findall('ThreadSize'):
        size = ts.find('Size').text
        pitch = ts.find('Designation/Pitch').text
        designation = ts.find('Designation/ThreadDesignation').text
        # Prüfen, ob leer, None oder null
        if (designation is None or designation.strip() == '' or
            size is None or size.strip() == '' or float(size) == 0.0 or
            pitch is None or pitch.strip() == '' or float(pitch) == 0.0):
            to_remove.append(ts)
    # Entfernen
    for ts in to_remove:
        root.remove(ts)
    tree.write(path)
    with open(path, 'r') as f:
        xml_str = f.read()
    pretty_xml = xml.dom.minidom.parseString(xml_str).toprettyxml(indent="  ")
    pretty_xml_clean = remove_blank_lines(pretty_xml)
    with open(path, 'w') as f:
        f.write(pretty_xml_clean)
    print(f"✅ {len(to_remove)} ungültige Einträge entfernt.")
    
    
def validate_thread_entry(path, designation_en):
    tree = ET.parse(path)
    root = tree.getroot()
    for ts in root.findall('ThreadSize'):
        designation = ts.find('Designation/ThreadDesignation').text
        if designation == designation_en:
            size = ts.find('Size').text
            pitch = ts.find('Designation/Pitch').text
            threads = ts.find('Designation').findall('Thread')
            gender_classes = [(t.find('Gender').text, t.find('Class').text) for t in threads]

            missing_genders = []
            for expected in [('external', '6g'), ('internal', '6H'), ('external', '4g6g')]:
                if expected not in gender_classes:
                    missing_genders.append(expected)

            print(f"🔍 Überprüfung von {designation_en}:")
            print(f"  Size: {size}")
            print(f"  Pitch: {pitch}")
            print(f"  Gefundene Gender/Class: {gender_classes}")

            if not size or not pitch:
                print("⚠️ Size oder Pitch fehlt oder ist leer!")
            if missing_genders:
                print(f"⚠️ Fehlende Gender/Class-Kombinationen: {missing_genders}")
            else:
                print("✅ Alle Pflichtfelder und Gender/Class-Kombinationen vorhanden.")
            return
    print(f"❌ Eintrag {designation_en} nicht gefunden.")

def load_dropdown_options():
    tree = ET.parse(xml_path)
    root = tree.getroot()
    options = ['<Neu laden>']
    for thread_size in root.findall('ThreadSize'):
        for designation in thread_size.findall('Designation'):
            thread_designation = designation.find('ThreadDesignation').text
            if thread_designation:
                options.append(thread_designation)
    thread_dropdown.options = options

def delete_thread_from_xml(designation_en):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    found = False
    for ts in root.findall('ThreadSize'):
        designations = ts.findall('Designation')
        for des in designations:
            if des.find('ThreadDesignation').text == designation_en:
                ts.remove(des)
                found = True
                break  # nur eine löschen
        # Wenn danach keine Designation mehr drin ist → ganzen Block löschen
        if len(ts.findall('Designation')) == 0:
            root.remove(ts)
    if found:
        tree.write(xml_path)
        print(f"🗑️ Gewinde {designation_en} gelöscht.")
    else:
        print(f"⚠️ Gewinde {designation_en} nicht gefunden.")

def on_delete_clicked(b):
    delete_thread_from_xml(thread_dropdown.value)
    load_dropdown_options()
    thread_dropdown.value = '<Neu laden>'
    
    


def fill_fields_from_selection(change):
    selection = change['new']
    if selection == '<Neu laden>':
        for w in all_widgets:
            w.value = '' if isinstance(w, widgets.Text) else 0
        selected_designation['value'] = None
        return
    
    selected_designation['value'] = selection  # Merke ursprüngliche Auswahl

    tree = ET.parse(xml_path)
    root = tree.getroot()
    for thread_size in root.findall('ThreadSize'):
        for designation in thread_size.findall('Designation'):
            thread_designation = designation.find('ThreadDesignation').text
            if thread_designation == selection:
                size.value = float(thread_size.find('Size').text)
                designation_de.value = designation.find('CTD').text
                designation_en.value = thread_designation
                pitch.value = float(designation.find('Pitch').text)
                for t in designation.findall('Thread'):
                    gender = t.find('Gender').text
                    cls = t.find('Class').text
                    major = float(t.find('MajorDia').text)
                    pitch_dia = float(t.find('PitchDia').text)
                    minor = float(t.find('MinorDia').text)
                    if gender == 'external' and cls == '6g':
                        major_dia_ext.value, pitch_dia_ext.value, minor_dia_ext.value = major, pitch_dia, minor
                    elif gender == 'internal' and cls == '6H':
                        major_dia_int.value, pitch_dia_int.value, minor_dia_int.value = major, pitch_dia, minor
                        tap_drill.value = float(t.find('TapDrill').text)
                    elif gender == 'external' and cls == '4g6g':
                        major_dia_4g6g.value, pitch_dia_4g6g.value, minor_dia_4g6g.value = major, pitch_dia, minor
                break  # <--- nur diese Bezeichnung befüllen

def add_thread_to_xml(path, size, designation_de, designation_en, pitch,
                      major_dia_ext, pitch_dia_ext, minor_dia_ext,
                      major_dia_int, pitch_dia_int, minor_dia_int,
                      major_dia_4g6g, pitch_dia_4g6g, minor_dia_4g6g,
                      tap_drill, old_designation=None):
    tree = ET.parse(path)
    root = tree.getroot()

    # Finde oder erstelle passenden ThreadSize-Knoten
    for ts in root.findall('ThreadSize'):
        if ts.find('Size').text == str(size):
            # Innerhalb des gefundenen Size-Blocks:
            # Alte Designation entfernen, falls vorhanden
            for des in ts.findall('Designation'):
                if old_designation and des.find('ThreadDesignation').text == old_designation:
                    ts.remove(des)
                elif not old_designation and des.find('ThreadDesignation').text == designation_en:
                    ts.remove(des)
            target_ts = ts
            break
    else:
        # Size-Block noch nicht vorhanden → neu anlegen
        target_ts = ET.SubElement(root, 'ThreadSize')
        ET.SubElement(target_ts, 'Size').text = str(size)

    # Neue Designation anhängen
    des_el = ET.SubElement(target_ts, 'Designation')
    ET.SubElement(des_el, 'ThreadDesignation').text = designation_en
    ET.SubElement(des_el, 'CTD').text = designation_de
    ET.SubElement(des_el, 'Pitch').text = str(pitch)

    for g, c, mj, pd, mn, tap in [
        ('external', '6g', major_dia_ext, pitch_dia_ext, minor_dia_ext, None),
        ('internal', '6H', major_dia_int, pitch_dia_int, minor_dia_int, tap_drill),
        ('external', '4g6g', major_dia_4g6g, pitch_dia_4g6g, minor_dia_4g6g, None)
    ]:
        th = ET.SubElement(des_el, 'Thread')
        ET.SubElement(th, 'Gender').text = g
        ET.SubElement(th, 'Class').text = c
        ET.SubElement(th, 'MajorDia').text = str(mj)
        ET.SubElement(th, 'PitchDia').text = str(pd)
        ET.SubElement(th, 'MinorDia').text = str(mn)
        if tap is not None:
            ET.SubElement(th, 'TapDrill').text = str(tap)

    # Datei speichern + schön formatieren
    tree.write(path)
    with open(path, 'r') as f:
        xml_str = f.read()
    pretty_xml = xml.dom.minidom.parseString(xml_str).toprettyxml(indent="  ")
    pretty_xml_clean = remove_blank_lines(pretty_xml)
    with open(path, 'w') as f:
        f.write(pretty_xml_clean)

    print(f"✅ Gewinde {designation_en} ({designation_de}) erfolgreich in {path} gespeichert und formatiert.")
    
# --- Widgets und Layouts ---    
field_layout = Layout(width='500px')
vbox_layout = Layout(width='550px', margin='0 40px 20px 0')
style = {'description_width': '350px'}

# Widgets für die Eingabefelder
thread_dropdown = widgets.Dropdown(description='Vorhandene Gewinde:', options=['<Neu laden>'], layout=field_layout, style=style)
size = widgets.FloatText(description='Nenndurchmesser (Size):', layout=field_layout, style=style)
designation_de = widgets.Text(description='Custom Thread designation:', layout=field_layout, style=style)
designation_en = widgets.Text(description='Technical designation:', layout=field_layout, style=style)
pitch = widgets.FloatText(description='Steigung (Pitch):', layout=field_layout, style=style)


# Widgets für die Gewinde-Parameter
label_ext_6g = widgets.HTML(value='<div style="text-align:right;"><b>External 6g</b></div>')
major_dia_ext, pitch_dia_ext, minor_dia_ext = [widgets.FloatText(description=d, layout=field_layout, style=style)
                                               for d in ['MajorDia:', 'PitchDia:', 'MinorDia:']]

label_int_6H = widgets.HTML(value='<div style="text-align:right;"><b>Internal 6H</b></div>')
major_dia_int, pitch_dia_int, minor_dia_int, tap_drill = [widgets.FloatText(description=d, layout=field_layout, style=style)
                                                          for d in ['MajorDia:', 'PitchDia:', 'MinorDia:', 'TapDrill:']]

label_ext_4g6g = widgets.HTML(value='<div style="text-align:right;"><b>External 4g6g</b></div>')
major_dia_4g6g, pitch_dia_4g6g, minor_dia_4g6g = [widgets.FloatText(description=d, layout=field_layout, style=style)
                                                  for d in ['MajorDia:', 'PitchDia:', 'MinorDia:']]

# Button-Widgets
save_button = widgets.Button(description='Speichern', style=style)
save_new_button = widgets.Button(description='Als neu speichern', style=style)
delete_button = widgets.Button(description='Löschen', style=style)

# Save-Buttons initial deaktivieren
save_button.disabled = True
save_new_button.disabled = True
delete_button.disabled = True

def on_save_clicked(b):
    original_designation = selected_designation['value']

    if not original_designation:
        print("⚠️ Kein Eintrag ausgewählt. Bitte zuerst aus Dropdown wählen.")
        return

    # CDT ignorieren → stattdessen aus designation_en übernehmen
    current_designation_en = designation_en.value
    current_designation_de = current_designation_en  # gleichsetzen

    # Speichern mit gleicher Technical und CDT Bezeichnung
    add_thread_to_xml(xml_path, size.value, current_designation_de, current_designation_en, pitch.value,
                      major_dia_ext.value, pitch_dia_ext.value, minor_dia_ext.value,
                      major_dia_int.value, pitch_dia_int.value, minor_dia_int.value,
                      major_dia_4g6g.value, pitch_dia_4g6g.value, minor_dia_4g6g.value,
                      tap_drill.value, old_designation=original_designation)

    load_dropdown_options()
    thread_dropdown.value = original_designation

    save_button.style.button_color = 'lightgreen'
    save_new_button.style.button_color = ''
    delete_button.style.button_color = ''
    delete_button.style.button_color = ''

def on_save_new_clicked(b):
    base_name = designation_en.value.rstrip('_neu')
    counter = 1
    new_designation = f"{base_name}_neu"
    existing_designations = [opt for opt in thread_dropdown.options if opt != '<Neu laden>']

    while new_designation in existing_designations:
        new_designation = f"{base_name}_neu{counter}"
        counter += 1

    # CDT gleichsetzen
    current_designation_de = new_designation

    designation_en.value = new_designation

    add_thread_to_xml(xml_path, size.value, current_designation_de, new_designation, pitch.value,
                      major_dia_ext.value, pitch_dia_ext.value, minor_dia_ext.value,
                      major_dia_int.value, pitch_dia_int.value, minor_dia_int.value,
                      major_dia_4g6g.value, pitch_dia_4g6g.value, minor_dia_4g6g.value,
                      tap_drill.value)

    load_dropdown_options()
    thread_dropdown.value = new_designation

    save_new_button.style.button_color = 'lightgreen'
    save_button.style.button_color = ''
    delete_button.style.button_color = ''
    

save_button.on_click(on_save_clicked)
save_new_button.on_click(on_save_new_clicked)
delete_button.on_click(on_delete_clicked)

button_box = VBox([save_button, save_new_button, delete_button])

all_widgets = [size, designation_de, designation_en, pitch,
               major_dia_ext, pitch_dia_ext, minor_dia_ext,
               major_dia_int, pitch_dia_int, minor_dia_int, tap_drill,
               major_dia_4g6g, pitch_dia_4g6g, minor_dia_4g6g]

top_row = HBox([VBox([thread_dropdown, size, designation_en, pitch], layout=vbox_layout), button_box])

# Lade Dropdown mit Fehlerbehandlung
try:
    load_dropdown_options()
except Exception as e:
    print(f"⚠️ Fehler beim Laden der Dropdown-Optionen: {e}")
    thread_dropdown.options = ['<Neu laden>']

# Setze Dropdown auf Startwert
thread_dropdown.value = '<Neu laden>'
thread_dropdown.observe(fill_fields_from_selection, names='value')



def on_export_clicked(b):
    subprocess.run(['./bin/sync_xml.sh', 'export'])
    clean_invalid_threads(xml_path)
    status_label.value = f"✅ Export aus Fusion abgeschlossen am {datetime.datetime.now().strftime('%d.%m.%Y %H:%M:%S')}"
    save_button.disabled = False
    save_new_button.disabled = False
    delete_button.disabled = False
    save_button.style.button_color = 'lightgreen'
    save_new_button.style.button_color = 'lightgreen'
    delete_button.style.button_color = 'lightgreen'
    thread_dropdown.add_class('highlighted')
    export_button.style.button_color = None  # Zurück auf neutral/grau
    
    
def on_import_clicked(b):
    subprocess.run(['./bin/sync_xml.sh', 'import'])
    clean_invalid_threads(xml_path)
    last_import_time[0] = datetime.datetime.now()
    update_status_label()
    status_label.value = f"✅ Import in Fusion abgeschlossen am {last_import_time[0].strftime('%d.%m.%Y %H:%M:%S')}"
    save_button.style.button_color = 'lightgreen'
    save_new_button.style.button_color = 'lightgreen'
    delete_button.style.button_color = 'lightgreen'
    thread_dropdown.add_class('highlighted')
    
def on_venv_clicked(b):
    subprocess.run(['./bin/create_venv.sh'])
    status_label.value = f"✅ .venv neu erstellt: {datetime.datetime.now().strftime('%d.%m.%Y %H:%M:%S')}"


export_button.on_click(on_export_clicked)
import_button.on_click(on_import_clicked)
venv_button.on_click(on_venv_clicked)

# Buttons initial auf Grün (Get XML) setzen
export_button.style.button_color = 'lightgreen'


# Letzter Importzeitpunkt
last_import_time = [None]

# Sprachumschalter anzeigen
display(toggle_lang_button)

# obere Eingabezeile und Button-Block
display(top_row)

# Anzeige-Teil
display(
    control_buttons,
    status_label,
)

# Gewinde-Bereiche anzeigen
display(
    label_ext_6g, major_dia_ext, pitch_dia_ext, minor_dia_ext,
    label_int_6H, major_dia_int, pitch_dia_int, minor_dia_int, tap_drill,
    label_ext_4g6g, major_dia_4g6g, pitch_dia_4g6g, minor_dia_4g6g
)

# Unsichtbarer Spacer am Ende der App
spacer = widgets.Box(layout=widgets.Layout(height='30px'))
display(spacer)

update_labels()




⚠️ Konfigurationsdatei config_fusion_threaddata_path.ini wurde neu angelegt.


HTML(value='<h1>Fusion 360 Gewindetool – XML-Editor mit Jupyter Notebook</h1>')

Button(description='🇩🇪 Auf Deutsch umschalten', style=ButtonStyle())

Label(value='✅ Pfad geladen: /Users/apos/Library/Application Support/Autodesk/webdeploy/production/e70c239bb1d…

HTML(value='<div style="text-align:right;"><b>External 6g</b></div>')

...

HTML(value='<div style="text-align:right;"><b>External 4g6g</b></div>')

FloatText(value=0.0, description='MajorDia:', layout=Layout(width='500px'), style=DescriptionStyle(description…

FloatText(value=0.0, description='PitchDia:', layout=Layout(width='500px'), style=DescriptionStyle(description…

FloatText(value=0.0, description='MinorDia:', layout=Layout(width='500px'), style=DescriptionStyle(description…

Box(layout=Layout(height='30px'))